## Testing out the PuLP linear programming package.  We'll work through some examples. Ultimately, I want to compare this to ORTools.

In [1]:
# Import modules

from __future__ import division

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tqdm import tqdm
from pulp import *

## The "miracle worker problem"
* A 'miracle' worker is trying to optimize the production of two medicines.  
* To create one unit of medicine 1, you need 3 units of herb A and 2 units of herb B.
* To create one unit of medicine 2, you need 4 units of herb A and 1 unit of herb B.
* Medicine 1 can heal a person by 25 units of health.
* Medicine 2 can heal a person by 20 units of health.
* You have 25 units of herb A and 10 units of herb B.
* What quantity of medicines 1 and 2 should you make to maximize how much you can heal the next patient?

In [2]:
# Create the 'prob' variable.  This will contain the problem data and definition.
prob = LpProblem("The mircale worker", LpMaximize)

In [4]:
# Define the variables.  The units of medicine 1 produced will be variable x, and the units of medicine 2 produced with be variable y.
# The variables here are going to be integers (so the output, which is the last argument below, is an interger type).
# The second argument is the lower bound (can't have negative amounts of medicine).
# The third argument is the upper bound (which could be anything here really).
x = LpVariable("medicine_1_units", 0, None, LpInteger)
y = LpVariable('medicine_2_units', 0, None, LpInteger)

In [6]:
# Define the objective function.  We want to maximize the amount we can heal, so per the problem definition it must be 25*x + 20*y = health restored.
# We define this by adding it to prob
prob += 25*x + 20*y, "health restored; to be maximized"

In [7]:
# Add the supply constraints for the herbs needed to make the medicines.
# If we produce x of medicine 1 and y of medicine 2, we need 3 units of herb A to make medicine 1 and 4 units of herb B to make medicine 2. So we use 3*x + 4*y units of herb A.
# Similarly, we use 2*x + y units of herb B.
# We have 25 units of herb A and 10 units of herb B in supply, so the constraints on supply are:
prob += 3*x + 4*y <= 25, "Herb A constraint"
prob += 2*x + y <= 10, "Herb B constraint"

In [8]:
# We can write out the problem with definitions to a .lp file. This was we can reload and solve the problem without having to redefine everything.
prob.writeLP("miracle_worker.lp")

In [9]:
# Run the solver (using the PuLP default)
prob.solve()

1

In [10]:
# Get the solver status
print("Status: {0}".format(LpStatus[prob.status]))

Status: Optimal


In [12]:
# So we've got the optimal solution.
# We can get back each variable with its optimal value
for v in prob.variables():
    print("{0} = {1}".format(v.name, v.varValue))

medicine_1_units = 3.0
medicine_2_units = 4.0


In [13]:
# So the best course of action is to produce 3 units of medicine 1 and 4 units of medicine 2.
# The maximum amount of health that can be restored (from producing the above amount of medicines 1 and 2) is:
print('Total health that can be restored = {0}'.format(value(prob.objective)))

Total health that can be restored = 155.0
